In [1]:
import networkx as nx
import math

from burn_tree import *

In [12]:
def burn_most_leaves_rooted(tree, verbose=False):
    activators = []
    marked = set()
    bound = math.ceil(math.sqrt(tree.order()))
    
    # First, root at a central vertex (min eccentricity)
    eccens = nx.algorithms.distance_measures.eccentricity(tree)
    root = min(eccens, key=eccens.get)
    
    i = 0
    while tree.order() > 0:
        if verbose:
            print("\nTree size", tree.order())
            print("Nodes:", tree.nodes())
            print("Edges:", tree.edges())
        # Consider all vertices v of height at most i - which N_i[v] covers the most leaves?
        leaves = get_leaves(tree, root=root)
        node_distances = shortest_path_lengths(tree)
        eccens = nx.algorithms.distance_measures.eccentricity(tree)
        
        # Get all vertices within distance i of a leaf
        near_leaves = set()
        for leaf in leaves:
            nhood = get_neighbourhood(tree, source=leaf, radius=i, node_distances=node_distances)
            for node in nhood:
                near_leaves.add(node)
        
        # Remove those that are not >= sqrt(n) dist from the root
        near_leaves2 = set()
        for node in near_leaves:
            if node_distances[root][node] >= bound:
                near_leaves2.add(node)
                
        # If no nodes far enough from the root, consider all nodes
        if len(near_leaves2) == 0:
            near_leaves2 = near_leaves
        
        # For all v we just got: take one with max leaves in its neighbourhood
        max_leaves = 0
        max_node = None
        max_nhood = None
        for node in near_leaves2:
            nhood = get_neighbourhood(tree, source=node, radius=i, node_distances=node_distances)
            
            num_leaves = 0
            for v in nhood:
                if v in leaves:
                    num_leaves += 1
            
            if num_leaves > max_leaves or (num_leaves == max_leaves and node_distances[root][node] > node_distances[root][max_node]):
                # Tie break with max depth
                max_node = node
                max_nhood = nhood
                max_leaves = num_leaves                  
            
        # Burn that vertex
        if max_node in activators:
            activators.remove(max_node)
        activators.insert(0, max_node)
        
        # Remove all vertices we can without disconnecting the graph           
        # Emulate a do-while loop here...
        temp_hood = set()
        while True:
            changed = False
            eccens = nx.algorithms.distance_measures.eccentricity(tree)
            
            for node in max_nhood:
                if not is_bridge(tree, node):
                    tree.remove_node(node)
                    changed = True
                else:
                    temp_hood.add(node)
            
            max_nhood = temp_hood # So that we don't try to remove nodes that don't exist anymore
            
            # Exit loop when we cannot remove any more nodes
            if not changed or tree.order() == 0:
                break
        
        '''
        sorted_eccens = sorted(eccens.items(), key=lambda kv: kv[1], reverse=True)
        for node, eccen in sorted_eccens:
            # Only remove nodes within the appropriate neighbourhood
            if node in max_nhood:
                temp = tree.copy()
                temp.remove_node(node)
                
                # Only remove if the the tree does not become disconnected
                if temp.order() == 0 or nx.algorithms.components.is_connected(temp):
                    if verbose:
                        print("Removing", node)
                    tree = temp
                elif verbose:
                    print("Not removing", node)
        '''
        
        i += 1
    
    return activators

tree = nx.Graph()
for i in range(5):
    tree.add_node(i, key=i)
edges = [(0, 1), (0, 2), (0, 3), (0, 4)]
tree.add_edges_from(edges)
print(list(tree.nodes(data=True)))

bs = burn_most_leaves_rooted(tree, verbose=True)
print("\nBurning sequence", bs)

[(0, {'key': 0}), (1, {'key': 1}), (2, {'key': 2}), (3, {'key': 3}), (4, {'key': 4})]

Tree size 5
Nodes: [0, 1, 2, 3, 4]
Edges: [(0, 1), (0, 2), (0, 3), (0, 4)]

Tree size 4
Nodes: [0, 2, 3, 4]
Edges: [(0, 2), (0, 3), (0, 4)]

Burning sequence [0, 1]


In [ ]:
import os
import math
from graph_utils import *

# Try burning all the trees in the ./trees directory
DIR_NAME = "./trees"
directory = os.fsencode(DIR_NAME)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        #print(filename)
        filepath = os.path.join(DIR_NAME, filename)
        with open(filepath, 'r') as file:
            adj_mat = create_adj_mat(filepath)
            tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
            #pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
            #nx.draw(tree, pos=pos, with_labels=True)
            
            burning_sequence = burn_most_leaves(tree)
            print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d} | {3:15} | {4:20}'.format(len(burning_sequence),
                                                                     tree.order(),
                                                                     math.ceil(math.sqrt(tree.order())),
                                                                     filename,
                                                                     str(burning_sequence)))

In [ ]:
def show_graph(filepath):
    adj_mat = create_adj_mat(filepath)
    tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
    pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
    nx.draw(tree, pos=pos, with_labels=True);

#show_graph('./trees/graph_626.mat')
show_graph('./trees/graph_30698.mat')

In [ ]:
#filename = './trees/graph_626.mat'
filename = './trees/graph_30698.mat'
with open(filename, 'r') as file:
    adj_mat = create_adj_mat(filename)
    tree = nx.convert_matrix.from_numpy_matrix(adj_mat)
    #pos = nx.nx_pydot.pydot_layout(tree, prog='dot')
    #nx.draw(tree, pos=pos, with_labels=True)

    burning_sequence = burn_most_leaves(tree, verbose=True)
    print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d} | {3:15} | {4:20}'.format(len(burning_sequence),
                                                             tree.order(),
                                                             math.ceil(math.sqrt(tree.order())),
                                                             filename,
                                                             str(burning_sequence)))

In [ ]:
from random import randint

# Generate random trees and burn them...
for i in range(100):
    n = randint(1, 125)
    rand_tree = nx.generators.trees.random_tree(n, seed=randint(0, 213218321321))
    burning_sequence = burn_most_leaves(rand_tree)
    print('b(G)<={0:2d} | n={1:3d} | ceil(sqrt(n))={2:3d} | {3:20}'.format(len(burning_sequence),
                                                                     rand_tree.order(),
                                                                     math.ceil(math.sqrt(rand_tree.order())),
                                                                     str(burning_sequence)))
    if len(burning_sequence) > math.ceil(math.sqrt(rand_tree.order())):
        print("Burning sequence > sqrt(n)")
        pos = nx.nx_pydot.pydot_layout(rand_tree, prog='dot')
        #nx.draw(rand_tree, pos=pos, with_labels=True)

In [ ]:
found = 0
while found < 1000:   
    n = randint(1, 500)
    rand_tree = nx.generators.trees.random_tree(n, seed=randint(0, 213218321321))

    burning_sequence = burn_most_leaves(rand_tree)
    upper_bound = math.ceil(math.sqrt(rand_tree.order()))
        
    if len(burning_sequence) > upper_bound:
        found += 1  
        print('b(G)<={0:2d} | n={1:2d} | ceil(sqrt(n))={2:2d}'.format(len(burning_sequence),
                                                                     rand_tree.order(),
                                                                     math.ceil(math.sqrt(rand_tree.order()))))